In [ ]:
#General Notes
#p# = stands for partial cells which may or may not be executed for the project
import pandas as pd
import joblib
from google.colab import drive
drive.mount('/content/drive')
##upload the dataset in your google drive

In [ ]:
## read the CSV file
##Enter your path here where you have uploaded the dataset in your google drive
ratings = pd.read_csv('/content/drive/My Drive/Colab Notebooks/AnimeDataset/rating.csv')
anime = pd.read_csv('/content/drive/My Drive/Colab Notebooks/AnimeDataset/anime.csv')
merged = ratings.merge(anime, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])
merged.rename(columns = {'rating_user':'user_rating'}, inplace = True)
merged.head()


In [ ]:
# For computing reasons limiting the dataframe length to 10,000 users
merged=merged[['user_id', 'name', 'user_rating']]
merged_sub= merged[merged.user_id <= 10000]
merged_sub.head()

In [ ]:
user_ratings = merged_sub.pivot_table(index=['user_id'],columns=['name'],values='user_rating')
user_ratings.head()

In [ ]:
##Removing anime which have less than 35 users who rated it to reduce the computation power neede. and fill remaining NaN with 0
user_ratings = user_ratings.dropna(thresh=35,axis=1).fillna(0)
user_ratings.head()

In [ ]:
##Building our Similarity Matrix using pearson method
item_similarity_df = user_ratings.corr(method='pearson')
item_similarity_df.head(50)

In [ ]:
#P#free up memory
#joblib.dump(item_similarity_df,"item_similarity_df")

In [ ]:
##Function to calulate the similar anime score
def get_similar_anime(Anime_name,user_rating):
    similar_score = item_similarity_df[Anime_name]*(user_rating-5)
    similar_score = similar_score.sort_values(ascending=False)
    
    return similar_score

In [ ]:
#P#Creating a list of all names of anime from the .csv file for getting names from partial name
all_anime_names = list(anime.name.values)

In [ ]:
#P#Function to get anime name from partial name
def get_id_from_partial_name(partial):
    for name in all_anime_names:
        if partial in name:
            print(name,all_anime_names.index(name))

In [ ]:
##Function to Enter the anime and its ratings for the user
def create_alist():
  test_user=[]
  n=int(input("Enter Number of Anime you want to enter in your list: "))
  for x in range (n):
    test_n=input("Enter the anime name: ")
    test_r=int(input("Enter your rating for the anime: "))
    test_var= test_n , test_r
    test_user.append(test_var)
  print('*'*100)
  return (test_user)


In [ ]:
#P#Use this to find the complete name of your anime series like in the dataset,Replace your incomplete name below
get_id_from_partial_name("Naruto")

In [ ]:
def display_recom():
  cflag=-5
  cflag=int(input("Enter '1' you want to enter your list directly in the console Or \nEnter '0' you wish to use the pre exisisting template in the code below \nWe recommend to use the pre existing template as it is convinient \nNote: Edit the template if you wish to use it before hand  "))
  if cflag ==1:
    user1= create_alist()
  elif cflag == 0:
    user1 = [("One Punch Man",7),("Tokyo Ghoul",6),("Gyakkyou Burai Kaiji: Ultimate Survivor",8),("Steins;Gate",8),
                  ("Death Note",10),("Boku dake ga Inai Machi",9),("Code Geass: Hangyaku no Lelouch",7.5),("Code Geass: Hangyaku no Lelouch R2",8.5),
                 ("Shigatsu wa Kimi no Uso",9),("Shingeki no Kyojin",9),("FLCL",4),("Mob Psycho 100",7),("91 Days",6),("Naruto",1),
                  ("Mirai Nikki",3),("One Outs",8),("Boku no Hero Academia",2),("Ping Pong The Animation",6),("No Game No Life",7.5) ,("Zankyou no Terror",6)]

  print("\n \n")
  print("*"*100)


  def check_seen(Anime,seen_anime):
      for item,rating in seen_anime:
          if item == Anime:
              return True
      
      return False
          
  similar_anime = pd.DataFrame()

  for Anime,rating in user1:
      similar_anime = similar_anime.append(get_similar_anime(Anime,rating),ignore_index=True)

  all_recommend = similar_anime.sum().sort_values(ascending=False)
  all_recommend=all_recommend.head(30)
  print("Recommendations for you are")
  print('*'*100)
  for Anime,score in all_recommend.iteritems():
      if not check_seen(Anime,user1):
          print(Anime)

In [ ]:
#Main Function
display_recom()